In [1]:
import boto3
from botocore.exceptions import ClientError
import sys
import requests
import re
import os
import pandas as pd
from sqlalchemy import create_engine
import datetime
import psycopg2
import geopandas as gpd




In [3]:
def validate_ip(ip):
    ip_regex = r'^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$'
    if re.match( ip_regex, ip):
        return True
    else:
        print("IP Address provided does not match required pattern.")
        return False


In [4]:
def update_db_with_ip(access_key, secret_access_key,
                      security_group_id, rule_description = f'Updated on {datetime.date.today()}',
                      ip_address = None, port = 5432):
    """
    A function a given security group for database access to a provided IPV4 Adress.
    """
    ec2 = boto3.client('ec2',
                       region_name = 'us-east-2',
                       aws_access_key_id = access_key,
                       aws_secret_access_key = secret_access_key)
    # Get current IP address
    if ip_address is None:
        ip_data = requests.get('https://ifconfig.me/ip')
        ip_text = ip_data.text
    else:
        ip_text = ip_address
    try:
        if validate_ip:
            pass
        else:
            raise
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise
    try:
        response = ec2.authorize_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {
                    'FromPort': 5432,
                    'IpProtocol': 'tcp',
                    'IpRanges': [
                        {
                            'CidrIp': ip_text+'/32',
                            'Description': rule_description,
                        },
                    ],
                    'ToPort': 5432,
                },
            ],
        )
        print(response)
    except ClientError as e:
        if e.response["Error"]["Code"] == "InvalidPermission.Duplicate":
            # ignore the target exception
            print(f"{ip_text} already open to {port}")
            pass
        else:
            print(e.response["Error"]["Code"])
            raise(e)
    except:
        print("Did not work")
        print("Unexpected error:", sys.exc_info()[0])
        raise(e)

In [5]:
access_key=os.environ['AWS_ACCESS_KEY']
secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY']
ip_data = requests.get('https://ifconfig.me/ip')
security_group_id = 'sg-0849656f'

In [6]:
 ec2 = boto3.client('ec2',
                       region_name = 'us-east-2',
                       aws_access_key_id = access_key,
                       aws_secret_access_key = secret_access_key)

In [7]:
update_db_with_ip(access_key, secret_access_key, security_group_id=security_group_id ,ip_address =ip_data.text, port = 5432)


200.113.234.187 already open to 5432


In [8]:
def get_posgres_connection():
    
    access_key = os.getenv("AWS_ACCESS_KEY")
    secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    username = os.getenv("USERNAME")
    ip_update_descrition = f"{username} -{datetime.date.today()}"
    
   
    db_name = os.getenv("PSQL_DB_NAME")
    db_user = os.getenv("USERNAME")
    db_password = os.getenv("PASSWORD")
    db_host = os.getenv("PSQL_DB_HOST")
    sql_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}')
    return sql_engine


In [9]:
connection = get_posgres_connection()

In [10]:
def generate_PCODE(depart):
     code = '00'+ str(depart)
     return 'HT'+ code[len(code)-2:]
   


In [11]:
def generate_PCODE2(vilcom):
     code = '00'+ str(depart)
     return code[len(code)-2:]

In [12]:
generate_PCODE(10)

'HT10'

In [13]:
def load_depart(spa=None,dept=None,boudaries_dep=None,pop_dep=None):

    # STEP 1
    # loads spa dataset
    spa = pd.read_csv('spa.csv')
    # replaces blank space by (_)
    spa['facdesc_1'] = spa['facdesc_1'].str.replace(' ','_')
    # computes dummies columns
    spa = pd.get_dummies(spa, columns=['facdesc_1'],prefix='',prefix_sep='')
    # sums the site health facilities
    dept = spa.groupby(['departn','depart'])['CENTRE_DE_SANTE_AVEC_LIT','DISPENSAIRE','HOPITAL'].sum()
    # renames departn column to ADM1_FR perform merging
    dept=dept.reset_index().rename(columns= {'departn':'ADM1_FR'})
    # generates PCODE
    dept['ADM1_PCODE'] = dept['depart'].apply(lambda x: generate_PCODE(x))

    # STEP 2
    # reads adm1 shapefiles
    boudaries_dep = gpd.read_file('boundaries/hti_admbnda_adm1_cnigs_20181129.shp')
    # sets the geometry column
    boudaries_dep.set_geometry('geometry')
    # selects features needed
    boudaries_dep=boudaries_dep[['ADM1_EN','ADM1_FR','ADM1_HT','ADM1_PCODE','geometry']]

    
    # STEP 3
    # reads population dataset
    pop_dep = pd.read_excel('datasets/hti_adminboundaries_tabulardata.xlsx',sheet_name='hti_pop2019_adm1')
    # selects feautures needed
    pop_dep = pop_dep[['adm1code','IHSI_UNFPA_2019','IHSI_UNFPA_2019_female','IHSI_UNFPA_2019_male']]
    # renames adm1code to ADM1_PCODE
    pop_dep =pop_dep.rename(columns ={"adm1code": "ADM1_PCODE"})

     # STEP 4
    boudaries_dep = pd.merge(boudaries_dep,dept,how ='left',left_on=['ADM1_PCODE'],right_on= ['ADM1_PCODE'])
    # renames column 
    boudaries_dep = boudaries_dep.rename(columns ={'ADM1_FR_x': 'ADM1_FR'})
    # selects features
    columns = ['ADM1_PCODE','ADM1_EN','ADM1_FR','ADM1_HT','geometry','CENTRE_DE_SANTE_AVEC_LIT','DISPENSAIRE','HOPITAL']
    boudaries_dep = boudaries_dep[columns]
    # merges all dataset
    boudaries_dep = pd.merge(boudaries_dep,pop_dep,how ='left',on='ADM1_PCODE')
    return boudaries_dep
     


In [14]:
gdf = load_depart()

In [15]:
def load_com(spa=None,dept=None,boudaries_dep=None,pop_dep=None):

    # STEP 1
    # loads spa dataset
    spa = pd.read_csv('spa.csv')
    # replaces blank space by (_)
    spa['facdesc_1'] = spa['facdesc_1'].str.replace(' ','_')
    # computes dummies columns
    spa = pd.get_dummies(spa, columns=['facdesc_1'],prefix='',prefix_sep='')
    # sums the site health facilities
    com  = spa.groupby(['depart','departn','vilcomn','vilcom'])['CENTRE_DE_SANTE_AVEC_LIT','DISPENSAIRE','HOPITAL'].sum().reset_index()
   
    # renames columns needed
    com= com.rename(columns={'departn':'ADM1_FR','vilcomn':'ADM2_FR'})
    # generats PCODE
    com['ADM2_PCODE'] = com['depart'].apply(lambda x: generate_PCODE(x))+ com['vilcom'].apply(lambda x: generate_PCODE2(x))
    # remove duplicates
    com =com.groupby(['ADM2_PCODE'])['CENTRE_DE_SANTE_AVEC_LIT','DISPENSAIRE','HOPITAL'].sum()
    com = com.reset_index()
   

    # STEP 2
    # reads adm2 shapefiles
    boudaries_com = gpd.read_file('boundaries/hti_admbnda_adm2_cnigs_20181129.shp')
    # sets the geometry column
    boudaries_com.set_geometry('geometry')
    # selects features needed
    boudaries_com = boudaries_com[['ADM2_PCODE','ADM1_EN','ADM1_FR','ADM2_EN','ADM2_FR','geometry']]

    
    # STEP 3
    # reads population dataset
    pop_com = pd.read_excel('datasets/hti_adminboundaries_tabulardata.xlsx',sheet_name='hti_pop2019_adm2')
    # selects feautures needed
    pop_com = pop_com[['adm2code','IHSI_UNFPA_2019','IHSI_UNFPA_2019_female','IHSI_UNFPA_2019_male']]
    # renames adm1code to ADM1_PCODE
    pop_com = pop_com.rename(columns ={"adm2code": "ADM2_PCODE"})




     # STEP 4
    spa_boundaries_com =pd.merge(boudaries_com,com,how ='left',left_on=['ADM2_PCODE'],right_on= ['ADM2_PCODE'])
   
    spa_boundaries_com = pd.merge(spa_boundaries_com,pop_com,how ='left',on=['ADM2_PCODE'])
    spa_boundaries_com.sort_values(by ='ADM2_PCODE')    

    return spa_boundaries_com
     

In [16]:
load_depart().head()

,ADM1_PCODE,ADM1_EN,ADM1_FR,ADM1_HT,geometry,CENTRE_DE_SANTE_AVEC_LIT,DISPENSAIRE,HOPITAL,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
0,HT05,Artibonite,Artibonite,Latibonit,"POLYGON ((-72.69573 19.81251, -72.69408 19.811...",17,74,11,2.047153e+06,1034065,1013085
1,HT06,Centre,Centre,Sant,"POLYGON ((-71.98404 19.33367, -71.98348 19.332...",12,19,5,8.455942e+05,413649,431947
2,HT08,Grande'Anse,Grande'Anse,Grandans,"MULTIPOLYGON (((-73.75492 18.64331, -73.75429 ...",5,29,4,4.810653e+05,229914,251152
3,HT10,Nippes,Nippes,Nip,"POLYGON ((-73.56737 18.58722, -73.56536 18.585...",7,19,3,3.515933e+05,167185,184409
4,HT03,North,Nord,Nò,"POLYGON ((-72.55529 19.87769, -72.55468 19.877...",15,26,18,1.159762e+06,589825,569932


In [17]:
conn = connection.connect()

In [18]:
conn

In [66]:
mspp_covid19_cases = pd.read_sql('SELECT * FROM mspp_covid19_cases',conn)
replace_pattern = {'Grand Anse ': 'Grande\'Anse','Grand Tota ':'Grand Total','Centre ': 'Centre','Artibonite ':'Artibonite','Guest ': 'Guest','Nippes ':'Nippes','Nord ':'Nord','Nord-Est ': 'Nord-Est','Nord-Ouest ':'Nord-Ouest','Ouest ': 'Ouest','Sud ': 'Sud','Sud-Est ':'Sud-Est'}
mspp_covid19_cases.departement=mspp_covid19_cases.departement.replace(replace_pattern)
mspp_covid19_cases.to_csv('datasets/mspp_covid19_cases.csv')


In [67]:
mspp_covid19_cases.head()

,departement,cas_suspects,cas_confirmes,deces,taux_de_letalite,document_date
0,Artibonite,47,4,0,0.0,2020-04-15
1,Centre,14,1,0,0.0,2020-04-15
2,Grande'Anse,2,0,0,0.0,2020-04-15
3,Nippes,16,2,0,0.0,2020-04-15
4,Nord,28,0,0,0.0,2020-04-15


In [78]:
my_list = []
for _,row in mspp_covid19_cases.iterrows():
    my_list.append(dict(departement	= row['departement'],cas_confirmes=row['cas_confirmes'],deces = row['deces'],taux_de_letalite = row['taux_de_letalite'],document_date =row['document_date'].value))
my_list

[{'departement': 'Artibonite',
  'cas_confirmes': 4,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': 1586908800000000000},
 {'departement': 'Centre',
  'cas_confirmes': 1,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': 1586908800000000000},
 {'departement': "Grande'Anse",
  'cas_confirmes': 0,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': 1586908800000000000},
 {'departement': 'Nippes',
  'cas_confirmes': 2,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': 1586908800000000000},
 {'departement': 'Nord',
  'cas_confirmes': 0,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': 1586908800000000000},
 {'departement': 'Nord-Est',
  'cas_confirmes': 7,
  'deces': 2,
  'taux_de_letalite': 0.29,
  'document_date': 1586908800000000000},
 {'departement': 'Nord-Ouest',
  'cas_confirmes': 1,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': 1586908800000000000},
 {'departement': 'Ouest',
  'cas_confirmes': 24,
  'deces': 1,
  'tau

In [85]:
def get_all_dept():
   mspp_covid19_cases = pd.read_csv('datasets/mspp_covid19_cases.csv')
   my_list = []
   for _,row in mspp_covid19_cases.iterrows():
    my_list.append(dict(departement	= row['departement'],cas_confirmes=row['cas_confirmes'],deces = row['deces'],taux_de_letalite = row['taux_de_letalite'],document_date =row['document_date']))
   my_list
   return my_list


In [86]:
get_all_dept()

[{'departement': 'Artibonite',
  'cas_confirmes': 4,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': '2020-04-15'},
 {'departement': 'Centre',
  'cas_confirmes': 1,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': '2020-04-15'},
 {'departement': "Grande'Anse",
  'cas_confirmes': 0,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': '2020-04-15'},
 {'departement': 'Nippes',
  'cas_confirmes': 2,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': '2020-04-15'},
 {'departement': 'Nord',
  'cas_confirmes': 0,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': '2020-04-15'},
 {'departement': 'Nord-Est',
  'cas_confirmes': 7,
  'deces': 2,
  'taux_de_letalite': 0.29,
  'document_date': '2020-04-15'},
 {'departement': 'Nord-Ouest',
  'cas_confirmes': 1,
  'deces': 0,
  'taux_de_letalite': 0.0,
  'document_date': '2020-04-15'},
 {'departement': 'Ouest',
  'cas_confirmes': 24,
  'deces': 1,
  'taux_de_letalite': 0.04,
  'document_date': '2020-04

In [5]:
import pandas as pd

In [6]:
tranlator = pd.read_csv('translation/translate1.csv')

In [15]:
tranlator.set_index('name',inplace =True)

In [17]:
tranlator.to_json('translation/translator.json')

In [18]:
tranlator

,en,kr,fr
name,,,
site_name,Health Systems Dashboard,Zouti Laswenyaj,NaN
home,Home,Paj Akèy,NaN
map,Interactive Map,Kat entèraktif,NaN
dashboard,Dashboard,Tablo,NaN
total_site,Number of Health Facilities,Kantite sant sante,NaN
dept,Department,Depatman,NaN
com,Commune,Komin,NaN
all_sites,Total Health Facilities,Tout Sant Sante yo,NaN
hospital,Hospital,Lopital,NaN
